* Section 1

  - <a href="#ScriptSpt1pt1">Script S.1.1: Spin--Boson Model Parameters</a>

  - <a href="#ScriptSpt1pt2">Script S.1.2: Using TT-TFD To Simulate The Spin--Boson Model</a>

* Section 2

  - <a href="#ScriptSpt2pt1">Script S.2.1: Projected Liouvillian</a>

  - <a href="#ScriptSpt2pt2">Script S.2.2: The Dynamics With Projected Liouvillian Only</a>

* Section 3

  - <a href="#ScriptSpt3pt1">Script S.3.1: Using QFlux To Obtain The Memory Kernel</a>

  - <a href="#ScriptSpt3pt2">Script S.3.2: The Propagator</a>

  - <a href="#ScriptSpt3pt3">Script S.3.3: Projection-Free Inputs $ {\cal F}(\tau) $ And $ \dot{\cal F}(\tau) $</a>

  - <a href="#ScriptSpt3pt4">Script S.3.4: Linear Term $ g(t) $</a>

  - <a href="#ScriptSpt3pt5">Script S.3.5: Memory Kernel - Volterra Algorithm</a>

  - <a href="#ScriptSpt3pt6">Script S.3.6: Function To Calculate Integral Via Trapezoidal Rule</a>

  - <a href="#ScriptSpt3pt7">Script S.3.7: Plot The Memory Kernel</a>

* Section 4

  - <a href="#ScriptSpt4pt1">Script S.4.1: Solve GQME Through QFlux</a>

  - <a href="#ScriptSpt4pt2">Script S.4.2: GQME - Propagation Via RK4 Method</a>

  - <a href="#ScriptSpt4pt3">Script S.4.3: Calculating The Function $ f $</a>

  - <a href="#ScriptSpt4pt4">Script S.4.4: GQME - Propagation Of The Density Matrix</a>

* Section 5

  - <a href="#ScriptSpt5pt1">Script S.5.1: Calculating $ {\cal{G}}(t) $ by solving the GQME</a>

  - <a href="#ScriptSpt5pt2">Script S.5.2: Dilation Of The Non-Unitary Propagator</a>

  - <a href="#ScriptSpt5pt3">Script S.5.3: Using QFlux To Perform Quantum Algorithm For GQME</a>

* Section 6

  - <a href="#ScriptSpt6pt1">Script S.6.1: Installing And Importing Qiskit Dependencies</a>

  - <a href="#ScriptSpt6pt2">Script S.6.2: Qasm  Simulation For GQME</a>

  - <a href="#ScriptSpt6pt3">Script S.6.3: Visualizing The Results</a>

# QFlux Installation

In [ ]:
!pip install qflux[gqme]

# Section 1

## Script S.1.1: Spin--Boson Model Parameters <a name="ScriptSpt1pt1"></a>

In [ ]:
GAMMA_DA = 1.0   # diabatic coupling
EPSILON = 1.0    # energy bias
BETA = 5.0       # inverse temperature
XI = 0.1         # Kondo parameter
OMEGA_C = 2.0    # cutoff frequency


## Script S.1.2: Using TT-TFD To Simulate The Spin--Boson Model <a name="ScriptSpt1pt2"></a>

In [ ]:
# Install qflux with:
# !pip install qflux[gqme]
import qflux
import qflux.GQME.readwrite as wr
import qflux.GQME.params as pa
import matplotlib.pyplot as plt
import os

# The TT-TFD simulation may take significant computational time.
# Precomputed results are provided and will be read automatically unless Is_run_dynamics = True.

Is_run_dynamics = True
data_path = "."

if Is_run_dynamics:
    import qflux.GQME.tt_tfd as tfd
    # RDO: reduced density operator containing populations and coherences
    # initial_state = 0 corresponds to the donor state
    t, RDO_arr = tfd.tt_tfd(initial_state=0, show_steptime=True, update_type='rk4')
    os.makedirs(data_path + "/GQME_Example/TTTFD_Output/", exist_ok=True)
    wr.output_operator_array(t, RDO_arr, data_path + "/GQME_Example/TTTFD_Output/TFDSigma_")

# Read precomputed data and plot
t, RDO_arr = wr.read_operator_array(pa.TIME_STEPS, data_path + "/GQME_Example/TTTFD_Output/TFDSigma_")
plt.figure(figsize=(6,2))
plt.plot(t, RDO_arr[:,0].real, 'b-', label='TT-TFD')
plt.xlabel(r'$\Gamma t$', fontsize=15)
plt.ylabel(r'$\sigma_{DD}(t)$', fontsize=15)
plt.legend()


# Section 2

## Script S.2.1: Projected Liouvillian <a name="ScriptSpt2pt1"></a>

In [ ]:
import numpy as np

LN0 = np.zeros((pa.DOF_E_SQ, pa.DOF_E_SQ))
LN0[0][1] = LN0[1][0] = LN0[2][3] = LN0[3][2] = -GAMMA_DA
LN0[0][2] = LN0[2][0] = LN0[1][3] = LN0[3][1] = GAMMA_DA
LN0[1][1] = 2. * EPSILON
LN0[2][2] = -2. * EPSILON


## Script S.2.2: The Dynamics With Projected Liouvillian Only <a name="ScriptSpt2pt2"></a>

In [ ]:
import scipy.linalg as LA

sigma_liou = np.zeros((pa.TIME_STEPS, pa.DOF_E_SQ), dtype=np.complex128)
time_arr = np.linspace(0,(pa.TIME_STEPS-1)*pa.DT,pa.TIME_STEPS)
sigma_liou[0] = np.array([1.0,0,0,0],dtype=np.complex128)
for i in range(1,pa.TIME_STEPS):
    sigma_liou[i] = LA.expm(-1j*LN0*pa.DT)@sigma_liou[i-1]

#read TT-TFD result and plot to compare
timeVec, sigma_tt_tfd = wr.read_operator_array(pa.TIME_STEPS, data_path + "/GQME_Example/TTTFD_Output/TFDSigma_")
plt.figure(figsize=(6,2))
plt.plot(time_arr, sigma_liou[:,0].real,'b-', label='Liouvillian only')
plt.plot(timeVec,  sigma_tt_tfd[:,0].real,'ko', markersize=4,markevery=15, label='TT-TFD')
plt.xlabel(r'$\Gamma t$',fontsize=15)
plt.ylabel(r'$\sigma_{DD}$(t)',fontsize=15)
_ = plt.legend(loc = 'upper right')


# Section 3

## Script S.3.1: Using QFlux To Obtain The Memory Kernel <a name="ScriptSpt3pt1"></a>

In [ ]:
from qflux.GQME.dynamics_GQME import DynamicsGQME

# Setup the Hamiltonian and initial state for Spin-Boson Model
Hsys = pa.EPSILON*pa.Z + pa.GAMMA_DA*pa.X
rho0 = np.zeros((pa.DOF_E,pa.DOF_E),dtype=np.complex128)
rho0[0,0] = 1.0

#Create the Spin-Boson model (SBM)
SBM = DynamicsGQME(pa.DOF_E,Hsys,rho0)
SBM.setup_timestep(pa.DT, pa.TIME_STEPS)

#The line below calculates all U elements with TT-TFD. The expected waiting time is 40 minutes on Google Colab.
#To save time, the results are already pre-computed and saved, and Is_run_dynamics is therefore set as False.
#The following code would still run normally. Please set Is_run_dynamics = True if one wishes to perform these calculations.
if Is_run_dynamics:
    print('==================now using tt-tfd to calculate propagator')
    timeVec,Gt = SBM.cal_propagator_tttfd()
    print('End of calculate propagator')

    #output the propagator
    os.makedirs(data_path + "/GQME_Example/U_Output/")
    wr.output_superoper_array(timeVec,Gt,data_path+"/GQME_Example/U_Output/U_")
else:
    timeVec,Gt = wr.read_superoper_array(pa.TIME_STEPS,data_path+"/GQME_Example/U_Output/U_")
    SBM.setup_propagator(Gt)

kernel = SBM.get_memory_kernel()


## Script S.3.2: The Propagator <a name="ScriptSpt3pt2"></a>

In [ ]:

def cal_U_tt_tfd():

    U = np.zeros((pa.TIME_STEPS, pa.DOF_E_SQ, pa.DOF_E_SQ), dtype=np.complex128)

    # tt-tfd with initial state 0,1,2,3
    # initial state |0> means donor state |D>, |3> means acceptor state |A>
    # |1> is (|D> + |A>)/sqrt(2), |2> is (|D> + i|A>)/sqrt(2)
    t,U[:,:,0] = tfd.tt_tfd(0)
    t,U[:,:,1] = tfd.tt_tfd(1)
    t,U[:,:,2] = tfd.tt_tfd(2)
    t,U[:,:,3] = tfd.tt_tfd(3)

    U_final = U.copy()

    # the coherence elements that start at initial state |D><A| and |A><D|
    # is the linear combination of above U results
    # |D><A| = |1><1| + i * |2><2| - 1/2 * (1 + i) * (|0><0| + |3><3|)
    U_final[:,:,1] = U[:,:,1] + 1.j * U[:,:,2] - 0.5 * (1. + 1.j) * (U[:,:,0] + U[:,:,3])

    # |A><D| = |1><1| - i * |2><2| - 1/2 * (1 - i) * (|0><0| + |3><3|)
    U_final[:,:,2] = U[:,:,1] - 1.j * U[:,:,2] - 0.5 * (1. - 1.j) * (U[:,:,0] + U[:,:,3])

    #output U
    os.makedirs(data_path + '/GQME_Example/U_Output/', exist_ok=True)
    wr.output_superoper_array(t,U_final, data_path + "/GQME_Example/U_Output/U_")

    return 0
#The line below calculates all U elements with TT-TFD. The expected waiting time on Google Colab is 40 minutes.
#To save time, the results are already pre-computed and saved, and Is_run_dynamics is set as False.
#The following code would still run normally. Please set Is_run_dynamics = True if one wishes to perform these calculations.
if Is_run_dynamics:
    cal_U_tt_tfd()


## Script S.3.3: Projection-Free Inputs $ {\cal F}(\tau) $ And $ \dot{\cal F}(\tau) $ <a name="ScriptSpt3pt3"></a>

In [ ]:
# the proj-free input from U data
def cal_F():
    # read the propagator data from files
    timeVec,U = wr.read_superoper_array(pa.TIME_STEPS, data_path + "/GQME_Example/U_Output/U_")

    F = np.zeros((pa.TIME_STEPS, pa.DOF_E_SQ, pa.DOF_E_SQ), dtype=np.complex128)
    Fdot = np.zeros((pa.TIME_STEPS, pa.DOF_E_SQ, pa.DOF_E_SQ), dtype=np.complex128)

    for j in range(pa.DOF_E_SQ):
        for k in range(pa.DOF_E_SQ):
            # extracts real and imag parts of U element
            Ureal = U[:,j,k].copy().real
            Uimag = U[:,j,k].copy().imag

            # F = i * d/dt U so Re[F] = -1 * d/dt Im[U] and Im[F] = d/dt Re[U]
            Freal = -1. * np.gradient(Uimag.flatten(), pa.DT, edge_order = 2)
            Fimag = np.gradient(Ureal.flatten(), pa.DT, edge_order = 2)

            # Fdot = d/dt F so Re[Fdot] = d/dt Re[F] and Im[Fdot] = d/dt Im[F]
            Fdotreal = np.gradient(Freal, pa.DT)
            Fdotimag = np.gradient(Fimag, pa.DT)

            F[:,j,k] = Freal[:] + 1.j * Fimag[:]
            Fdot[:,j,k] = Fdotreal[:] + 1.j * Fdotimag[:]

    # write the result to the file
    os.makedirs(data_path + "/GQME_Example/ProjFree_Output/", exist_ok=True)
    wr.output_superoper_array(timeVec,F,data_path + "/GQME_Example/ProjFree_Output/F_")
    wr.output_superoper_array(timeVec,Fdot,data_path + "/GQME_Example/ProjFree_Output/Fdot_")

    return timeVec,F,Fdot

timeVec,F,Fdot = cal_F()


## Script S.3.4: Linear Term $ g(t) $ <a name="ScriptSpt3pt4"></a>

In [ ]:
linearTerm = 1.j * Fdot.copy() # first term of the linear part
for l in range(pa.TIME_STEPS):
    # subtract second term of linear part
    linearTerm[l,:,:] -= 1./pa.HBAR * F[l,:,:] @ LN0


## Script S.3.5: Memory Kernel - Volterra Algorithm <a name="ScriptSpt3pt5"></a>

In [ ]:
def CalculateIntegral(DOF_E_SQ, F, linearTerm, prevKernel, kernel, TIME_STEPS=pa.TIME_STEPS, DT=pa.DT):

    # time step loop starts at 1 because K is equal to linear part at t = 0
    for n in range(1, TIME_STEPS):
        kernel[n,:,:] = 0.

        # f(a) and f(b) terms
        kernel[n,:,:] += 0.5 * DT * F[n,:,:] @ kernel[0,:,:]
        kernel[n,:,:] += 0.5 * DT * F[0,:,:] @ prevKernel[n,:,:]

        # sum of f(a + kh) term
        for c in range(1, n):
            # since a new (supposed-to-be-better) guess for the
            # kernel has been calculated for previous time steps,
            # can use it rather than prevKernel
            kernel[n,:,:] += DT * F[n - c,:,:] @ kernel[c,:,:]

        # multiplies by i and adds the linear part
        kernel[n,:,:] = 1.j * kernel[n,:,:] + linearTerm[n,:,:]

    return kernel

In [ ]:
import time

START_TIME = time.time() # starts timing
# sets initial guess to the linear part
prevKernel = linearTerm.copy()
kernel = linearTerm.copy()

# loop for iterations
for numIter in range(1, pa.MAX_ITERS + 1):

    iterStartTime = time.time() # starts timing of iteration
    print("Iteration:", numIter)

    # calculates kernel using prevKernel and trapezoidal rule
    kernel = CalculateIntegral(pa.DOF_E_SQ, F, linearTerm, prevKernel, kernel)

    numConv = 0 # parameter used to check convergence of entire kernel
    for i in range(pa.DOF_E_SQ):
        for j in range(pa.DOF_E_SQ):
            for n in range(pa.TIME_STEPS):
                # if matrix element and time step of kernel is converged, adds 1
                if abs(kernel[n][i][j] - prevKernel[n][i][j]) <= pa.CONVERGENCE_PARAM:
                    numConv += 1

                # if at max iters, prints which elements and time steps did not
                # converge and prevKernel and kernel values
                elif numIter == pa.MAX_ITERS:
                    print("\tK time step and matrix element that didn't converge: %s, %s%s"%(n,i,j))

    print("\tIteration time:", time.time() - iterStartTime)

    # enters if all times steps and matrix elements of kernel converged
    if numConv == pa.TIME_STEPS * pa.DOF_E_SQ * pa.DOF_E_SQ:
        # prints number of iterations and time necessary for convergence
        print("Number of Iterations:", numIter, "\tVolterra time:", time.time() - START_TIME)

        # prints memory kernel to files
        os.makedirs(data_path + "/GQME_Example/K_Output/", exist_ok=True)
        wr.output_superoper_array(timeVec,kernel,data_path + "/GQME_Example/K_Output/K_")

        break # exits the iteration loop

    # if not converged, stores kernel as prevKernel, zeros the kernel, and then
    # sets kernel at t = 0 to linear part
    prevKernel = kernel.copy()
    kernel = linearTerm.copy()

    # if max iters reached, prints lack of convergence
    if numIter == pa.MAX_ITERS:
        print("\tERROR: Did not converge for %s iterations"%pa.MAX_ITERS)
        print("\tVolterra time:", print(time.time() - START_TIME))


## Script S.3.6: Function To Calculate Integral Via Trapezoidal Rule <a name="ScriptSpt3pt6"></a>

In [ ]:
def CalculateIntegral(DOF_E_SQ, F, linearTerm, prevKernel, kernel, TIME_STEPS=pa.TIME_STEPS, DT=pa.DT):

    # time step loop starts at 1 because K is equal to linear part at t = 0
    for n in range(1, TIME_STEPS):
        kernel[n,:,:] = 0.

        # f(a) and f(b) terms
        kernel[n,:,:] += 0.5 * DT * F[n,:,:] @ kernel[0,:,:]
        kernel[n,:,:] += 0.5 * DT * F[0,:,:] @ prevKernel[n,:,:]

        # sum of f(a + kh) term
        for c in range(1, n):
            # since a new (supposed-to-be-better) guess for the
            # kernel has been calculated for previous time steps,
            # can use it rather than prevKernel
            kernel[n,:,:] += DT * F[n - c,:,:] @ kernel[c,:,:]

        # multiplies by i and adds the linear part
        kernel[n,:,:] = 1.j * kernel[n,:,:] + linearTerm[n,:,:]

    return kernel

## Script S.3.7: Plot The Memory Kernel <a name="ScriptSpt3pt7"></a>

In [ ]:
# plot the kernel without the last two boundary points that have numerical errors
plt.figure(figsize=(6,2))
plt.plot(timeVec[:-2], kernel[:-2,1,0].real,'b-', label=r'Re $\mathcal{K}_{DA,DD}$')
plt.plot(timeVec[:-2], kernel[:-2,0,0].real,'k-', label=r'Re $\mathcal{K}_{DD,DD}$')
plt.xlabel(r'$\Gamma t$',fontsize=15)
plt.ylabel(r'$\mathcal{K}$(t)',fontsize=15)
plt.legend(loc = 'upper right')


# Section 4

## Script S.4.1: Solve GQME Through QFlux <a name="ScriptSpt4pt1"></a>

In [ ]:
sigma = SBM.solve_gqme(kernel, pa.MEM_TIME)


## Script S.4.2: GQME - Propagation Via RK4 Method <a name="ScriptSpt4pt2"></a>

In [ ]:
def PropagateRK4(currentTime, memTime, kernel,
                 sigma_hold, sigma, DT=pa.DT):

    f_0 = Calculatef(currentTime, memTime,
                     kernel, sigma, sigma_hold)

    k_1 = sigma_hold + DT * f_0 / 2.
    f_1 = Calculatef(currentTime + DT / 2., memTime,
                     kernel, sigma, k_1)

    k_2 = sigma_hold + DT * f_1 /2.
    f_2 = Calculatef(currentTime + DT / 2., memTime,
                     kernel, sigma, k_2)

    k_3 = sigma_hold + DT * f_2
    f_3 = Calculatef(currentTime + DT, memTime,
                     kernel, sigma, k_3)

    sigma_hold += DT / 6. * (f_0 + 2. * f_1 + 2. * f_2 + f_3)

    return sigma_hold


## Script S.4.3: Calculating The Function $ f $ <a name="ScriptSpt4pt3"></a>

In [ ]:
def Calculatef(currentTime, memTime, kernel, sigma_array, kVec, DT=pa.DT, HBAR=pa.HBAR, LN0=LN0):

    memTimeSteps = int(memTime / DT)
    currentTimeStep = int(currentTime / DT)

    f_t = np.zeros(kVec.shape, dtype=np.complex128)

    f_t -= 1.j / HBAR * LN0 @ kVec

    limit = memTimeSteps
    if currentTimeStep < (memTimeSteps - 1):
        limit = currentTimeStep
    for l in range(limit):
        f_t -= DT * kernel[l,:,:] @ sigma_array[currentTimeStep - l]

    return f_t


## Script S.4.4: GQME - Propagation Of The Density Matrix <a name="ScriptSpt4pt4"></a>

In [ ]:
# read the memory kernel
timeVec, kernel = wr.read_superoper_array(len(t), data_path + "/GQME_Example/K_Output/K_")

# array for reduced density matrix elements
sigma = np.zeros((pa.TIME_STEPS, pa.DOF_E_SQ), dtype=np.complex128)
# array to hold copy of sigma
sigma_hold = np.zeros(pa.DOF_E_SQ, dtype = np.complex128)

# sets the initial state at Donor State
sigma[0,0] = 1.
sigma_hold[0] = 1.

# loop to propagate sigma
print(">>> Starting GQME propagation, memory time =", pa.MEM_TIME)
for l in range(pa.TIME_STEPS - 1): # it propagates to the final time step
    if l%100==0: print(l)
    currentTime = l * pa.DT

    sigma_hold = PropagateRK4(currentTime, pa.MEM_TIME, kernel, sigma_hold, sigma)

    sigma[l + 1] = sigma_hold.copy()

# prints sigma to files
os.makedirs(data_path + 'GQME_Output/', exist_ok=True)
wr.output_operator_array(timeVec, sigma, data_path + "GQME_Output/Sigma_")

# Read the reference data and plot
timeVec, sigma_tt_tfd = wr.read_operator_array(len(timeVec), data_path + "/GQME_Example/TTTFD_Output/TFDSigma_")
timeVec, sigma = wr.read_operator_array(len(timeVec), data_path + "GQME_Output/Sigma_")
plt.figure(figsize=(6,2))
plt.plot(timeVec, sigma[:,0],'b-', label='GQME')
plt.plot(timeVec, sigma_tt_tfd[:,0] ,'ko', markersize=4, markevery=60, label='benchmark_TT-TFD')
plt.xlabel(r'$\Gamma t$',fontsize=15)
plt.ylabel(r'$\sigma_{DD}$(t)',fontsize=15)
plt.legend()


# Section 5

## Script S.5.1: Calculating $ {\cal{G}}(t) $ by solving the GQME <a name="ScriptSpt5pt1"></a>

In [ ]:
# read the memory kernel
timeVec, kernel = wr.read_superoper_array(len(timeVec), data_path + "/GQME_Example/K_Output/K_")

# array for Propagator superoperator elements
G_prop = np.zeros((pa.TIME_STEPS, pa.DOF_E_SQ, pa.DOF_E_SQ), dtype=np.complex128)

# time 0 propagator: identity superoperator
G_prop[0] = np.eye(pa.DOF_E_SQ)
# array to hold copy of G propagator
G_prop_hold = np.eye((pa.DOF_E_SQ), dtype=np.complex128)

# loop to propagate G_prop using GQME
print(">>> Starting GQME propagation, memory time =", pa.MEM_TIME)
for l in range(pa.TIME_STEPS - 1): # it propagates to the final time step
    if l%100==0: print(l)
    currentTime = l * pa.DT

    G_prop_hold = PropagateRK4(currentTime, pa.MEM_TIME, kernel, G_prop_hold, G_prop)

    G_prop[l + 1] = G_prop_hold.copy()


## Script S.5.2: Dilation Of The Non-Unitary Propagator <a name="ScriptSpt5pt2"></a>

In [ ]:
from numpy import linalg as la
import scipy.linalg as sp

def dilate(array):

    # Normalization factor of 1.5 to ensure contraction
    norm = la.norm(array,2)*1.5
    array_new = array/norm

    ident = np.eye(array.shape[0])

    # Calculate the conjugate transpose of the G propagator
    fcon = (array_new.conjugate()).T

    # Calculate the defect matrix for dilation
    fdef = sp.sqrtm(ident - np.dot(fcon, array_new))

    # Calculate the defect matrix for the conjugate of the G propagator
    fcondef = sp.sqrtm(ident - np.dot(array_new, fcon))

    # Dilate the G propagator to create a unitary operator
    array_dilated = np.block([[array_new, fcondef], [fdef, -fcon]])

    return array_dilated, norm


## Script S.5.3: Using QFlux To Perform Quantum Algorithm For GQME <a name="ScriptSpt5pt3"></a>

In [ ]:
G_prop = SBM.solve_gqme(kernel, pa.MEM_TIME, dtype='Propagator')

from qflux.open_systems.quantum_simulation import QubitDynamicsOS

qSBM = QubitDynamicsOS(rep='Density', Nsys = pa.DOF_E, Hsys = Hsys, rho0 = rho0)
qSBM.set_count_str(['000','011'])
qSBM.set_dilation_method('Sz-Nagy')

pop_qc = qSBM.qc_simulation_vecdens(timeVec,Gprop=G_prop)


# Section 6

## Script S.6.1: Installing And Importing Qiskit Dependencies <a name="ScriptSpt6pt1"></a>

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Operator


## Script S.6.2: Qasm  Simulation For GQME <a name="ScriptSpt6pt2"></a>

In [ ]:
# Create a dictionary to store the measurement results
result = {'000': 0, '001': 0, '010': 0, '011': 0, '100': 0, '101': 0, '110': 0, '111': 0}

# Create lists to store the population for the acceptor and donor states
pop_accept = []
pop_donor = []

# initial state in the dilated space
rho0_dilated = np.concatenate((np.array([1 + 0j, 0, 0, 0]),np.zeros(pa.DOF_E_SQ)))

for i in range(pa.TIME_STEPS):

    qr = QuantumRegister(3)  # Create a quantum register with 3 qubits
    cr = ClassicalRegister(3)  # Create a classical register to store measurement results
    qc = QuantumCircuit(qr, cr)  # Combine the quantum and classical registers to create the quantum circuit

    # Initialize the quantum circuit with the initial state
    qc.initialize(rho0_dilated, qr)

    # Dilated propagator
    U_G, norm = dilate(G_prop[i])

    # Create a custom unitary operator with the dilated propagator
    U_G_op = Operator(U_G)

    # Apply the unitary operator to the quantum circuit's qubits
    qc.unitary(U_G_op, qr)
    # Measure the qubits and store the results in the classical register
    qc.measure(qr, cr)

    #Run the Simulation and Plot the Results
    simulator = QasmSimulator()
    shots = 2000  # Number of shots
    job = simulator.run(qc,shots=shots)
    counts = job.result().get_counts(qc)

    # Update the result dictionary
    for x in counts:
        result[x] = counts[x]

    # Calculate the populations of donor and acceptor states from measurement probabilities
    pdon = np.sqrt(result['000'] / shots) * norm  # Multiply by the normalization factor
    pacc = np.sqrt(result['011'] / shots) * norm  # Multiply by the normalization factor

    pop_donor.append(pdon)  # Stacking the population for the donor state
    pop_accept.append(pacc)  # Stacking the population for the acceptor state


## Script S.6.3: Visualizing The Results <a name="ScriptSpt6pt3"></a>

In [ ]:
# Read the exact TT-TFD results
timeVec, sigma_tt_tfd = wr.read_operator_array(pa.TIME_STEPS, data_path + "/GQME_Example/TTTFD_Output/TFDSigma_")
# Plot the population of the donor and acceptor states
plt.figure(figsize=(6,2))
plt.plot(timeVec, pop_donor, 'r-', label="quantum simulation")
plt.plot(timeVec, sigma_tt_tfd[:,0].real ,'ko', markersize=4, markevery=15, label='benchmark_TT-TFD')
plt.xlabel(r'$\Gamma t$',fontsize=15)
plt.ylabel(r'$\sigma_{DD}$(t)',fontsize=15)
plt.legend(loc = 'upper right')
